In [1]:
import sys
# from graph_dataset.dataset import GraphDataset
from rgcn_dataset import GraphDataset
from model.tripartite import GNNPolicy
from utilities import *
import torch
import torch_geometric
import os 
import time
from torch.optim.lr_scheduler import CosineAnnealingLR
import gurobipy as gp
import json
from torch_geometric.data import HeteroData

from pytorch_metric_learning import losses
from pytorch_metric_learning.distances import DotProductSimilarity

In [2]:
#set folder
instance = '80_c11_based_8_std'
model = 'tri_cl'
uc_model = '3bin'



model_save_path = f"./model_save/local_search_{uc_model}_{instance.split('_')[0]}_{model}/"
log_save_path =   f"./model_save/local_search_{uc_model}_{instance.split('_')[0]}_{model}_log/"
os.makedirs(model_save_path, exist_ok=True)
os.makedirs(log_save_path, exist_ok=True)

log_file = open(f'{log_save_path}train.log', 'wb')

In [3]:
LEARNING_RATE = 1e-5
NB_EPOCHS =1000
BATCH_SIZE = 1
NUM_WORKERS = 0
ETA_MIN=1e-6

In [4]:
train_data = GraphDataset(f'./datasets/{instance}/small_range_local_search_tripartite/train/{uc_model}/')
valid_data = GraphDataset(f'./datasets/{instance}/small_range_local_search_tripartite/valid/{uc_model}/')
train_iter = torch_geometric.loader.DataLoader(
    train_data, batch_size = BATCH_SIZE, shuffle = True,
    num_workers = NUM_WORKERS, pin_memory=True)
valid_iter = torch_geometric.loader.DataLoader(
    valid_data, batch_size = BATCH_SIZE, shuffle = False,
    num_workers = NUM_WORKERS)

In [5]:
# DEVICE = torch.device("cpu")
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
PredictModel = GNNPolicy().to(DEVICE)
# if os.path.exists(model_save_path+'model_best_con.pth'):
#     state_dict = torch.load(model_save_path+'model_best_con.pth')
#     PredictModel.load_state_dict(state_dict, strict=False)
#     print("载入模型成功")

In [6]:
def train(predict, data_loader, optimizer=None):
    """
    This function will process a whole epoch of training or validation, depending on whether an optimizer is provided.
    """
    infoNCE_loss_function = losses.NTXentLoss(temperature=0.07,distance=DotProductSimilarity()).to(DEVICE)
    # loss_fn = torch.nn.BCELoss()
    if optimizer:
        predict.train()
    else:
        predict.eval()
    mean_loss = 0
    n_samples_processed = 0
    with torch.set_grad_enabled(optimizer is not None):
        for step, batch in enumerate(data_loader):

            batch = batch.to(DEVICE)
            embeddings = predict(batch).reshape(len(batch), -1)
            # print(torch.isnan(embeddings).any())
            # print(batch.positive)
            # print(len(batch.positive),len(batch.positive[0]))
            
           
            anchor_positive = torch.tensor([i for i in range(len(batch)) for j in range(len(batch.positive[i]))])
            anchor_negative = torch.tensor([i for i in range(len(batch)) for j in range(len(batch.negative[i]))])
            positive_idx = torch.arange(len(batch),len(batch)+len(anchor_positive))
            negative_idx = torch.arange(len(batch)+len(anchor_positive),len(batch)+len(anchor_positive)+len(anchor_negative))
            
            # print(anchor_positive)
            # print(anchor_negative)
            # print(positive_idx)
            # print(negative_idx)
            positive = torch.from_numpy(np.concatenate(batch.positive)).to(DEVICE)
            negative = torch.from_numpy(np.concatenate(batch.negative)).to(DEVICE)
            # positive_idx = range(1,1+len(batch.positive))
            # negative_idx = range(1+len(batch.positive),1+len(batch.positive)+len(batch.negative))
            embeddings = torch.cat((embeddings, positive, negative),0)
            # print(embeddings.shape)
            triplets = (anchor_positive.to(DEVICE), positive_idx.to(DEVICE), anchor_negative.to(DEVICE), negative_idx.to(DEVICE))
            # triplets = (anchor_positive.clone().detach().to(DEVICE), positive_idx.clone().detach().to(DEVICE), anchor_negative.clone().detach().to(DEVICE), negative_idx.clone().detach().to(DEVICE))
            
            # break
            loss = infoNCE_loss_function(embeddings, indices_tuple = triplets)
            # loss = loss_fn(embeddings, average_local)
            if optimizer is not None:
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            mean_loss += loss.item()
            n_samples_processed += 1
    mean_loss /= n_samples_processed
    return mean_loss


In [7]:
optimizer = torch.optim.AdamW(PredictModel.parameters(), lr=LEARNING_RATE,weight_decay=1e-6)
scheduler = CosineAnnealingLR(optimizer, T_max=NB_EPOCHS//10, eta_min=ETA_MIN)
best_val_loss = 10
cur_epoch = 0


In [8]:
for epoch in range(NB_EPOCHS):
    begin=time.time()
    cur_epoch+=1
    train_loss = train(PredictModel, train_iter, optimizer)
    print(f"Epoch {epoch} Train loss: {train_loss:0.8f}",end='\t')
    valid_loss = train(PredictModel, valid_iter, None)
    print(f"Epoch {epoch} Valid loss: {valid_loss:0.8f}")
    if valid_loss<best_val_loss:
        cur_epoch=0
        best_val_loss = valid_loss
        print("best_loss",best_val_loss)
        torch.save(PredictModel.state_dict(),model_save_path+'model_best_con.pth')
    if epoch%2==0:
        torch.save(PredictModel.state_dict(),model_save_path+f'model_{epoch}.pth')
    if cur_epoch>100:
        break
    date_str = time.strftime('%Y-%m-%d  %H:%M:%S', time.localtime())
    st = f'@epoch{epoch}   Train loss:{train_loss}   Valid loss:{valid_loss}    TIME:{time.time()-begin}    DONE_TIME:{date_str}\n'
    log_file.write(st.encode())
    log_file.flush()
    scheduler.step()
print('done')

Epoch 0 Train loss: 2.90059888	Epoch 0 Valid loss: 2.91439615
best_loss 2.9143961536623264
Epoch 1 Train loss: 2.89329096	Epoch 1 Valid loss: 2.91374484
best_loss 2.9137448432671764
Epoch 2 Train loss: 2.89314052	Epoch 2 Valid loss: 2.91366404
best_loss 2.9136640425146063
Epoch 3 Train loss: 2.89308982	Epoch 3 Valid loss: 2.91391444
Epoch 4 Train loss: 2.89296066	Epoch 4 Valid loss: 2.91307579
best_loss 2.9130757888738255
Epoch 5 Train loss: 2.89279079	Epoch 5 Valid loss: 2.91296633
best_loss 2.912966327945681
Epoch 6 Train loss: 2.89266926	Epoch 6 Valid loss: 2.91398143
Epoch 7 Train loss: 2.89253221	Epoch 7 Valid loss: 2.91323461
Epoch 8 Train loss: 2.89240698	Epoch 8 Valid loss: 2.91315070
Epoch 9 Train loss: 2.89239079	Epoch 9 Valid loss: 2.91286822
best_loss 2.9128682224419866
